In [7]:
#LIBS to be used 

from MDPlus.core import Fasu, Cofasu
from MDPlus.analysis import pca
import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline

#getting details automatically from files
import glob
import csv
import pandas as pd
import time
import datetime

#saving data 
import pickle

import mdtraj as md

from MDPlus import fastfitting

In [30]:
def set_up_vars(atm,folder):
    print " "
    print " set up vars"
    strpath=atm+folder+"/rp.sess*/"+'zip.rp*'
    print strpath
    path_to_zip = glob.glob(strpath)
    print path_to_zip
    print type(path_to_zip)
    print path_to_zip.split("/")[-1]
    print path_to_zip[0].split("/")[-1].split("-pilot.0000")

    rpname = str(path_to_zip[0].split("/")[-1].split("-pilot.0000")[0])
    zipfolder = rpname+ '-pilot.0000/'
    prj = folder + zipfolder
    path   = atm+prj
    pathSt = atm+prj+"staging"
    pathCu = atm+prj+"cu009"
    #print path


    #print " "
    #print "path to cu"
    path_to_cu = glob.glob(atm+folder+zipfolder+"/cu009/"+"min-*_9.log")
    #print path_to_cu
    #print len(path_to_cu)

    iters = int(len(path_to_cu))+2-1 #need the -1 bec it failed before completion so counter is off
    #print 'iters =', iters
    return path_to_zip, rpname, zipfolder, path, pathSt, pathCu, iters, path_to_cu; 

###################################################3
def get_gmxcoco_data(atm,folder):
    #print  "#              gmxcoco.wcfg details        #"
    wsplit = []
    path_to_gmxcoco=atm+folder+"/gmxcoco.wcfg"    
    e=open(path_to_gmxcoco,"r")
    #print e.readlines()
    w = e.readlines()
    e.close()
    gmxcocolines = [5, 7, 12, 28, 31  ]
    for i in range(len(gmxcocolines)):
        #print w[gmxcocolines[i]]
        #print w[gmxcocolines[i]].split()
        wsplit.append(w[gmxcocolines[i]].split())
    cu = int(wsplit[1][2])
    return cu, path_to_gmxcoco ; 

####################################################
def get_run1mdp_data(atm,folder):
    path_to_run1=atm+folder+ "inp_files/"+"run1.mdp"    
    e=open(path_to_run1,"r")
    #print e.readlines()
    w = e.readlines()
    e.close()
    #for i in range(len(w)):
        #print i
        #print w[i]
    #lines = [9] #, 7, 12, 28, 31  ]
    #print w[4]
    cow = w[4].split("_")
    #print cow
    #print cow[-1]
    posre_k12 = cow[-1].strip()
    #print posre_k12
    #print w[9]
    cow = w[9].split()
    r1_em_step = cow[-1]
    #print r1_em_step
    #print w[10]
    cow = w[10].split()
    r1_n_steps = cow[-1]
    #print r1_n_steps

    run1 = []
    run1.append(w[9].split())
    run1.append(w[10].split())
    #print run1[0]
    #print run1[1]
    eqibTimePerIter = float(run1[0][2])*float(run1[1][2])
    #print 'total equilibrum time =', float(run1[0][2])*float(run1[1][2])
    run1mdp = []
    run1mdp.append(run1[0][2])
    run1mdp.append(run1[1][2])
    run1mdp.append(eqibTimePerIter)
    run1mdp.append(posre_k12)
    print run1mdp
    return run1mdp;

###############################################
def get_run2mdp_data(atm,folder):

    ##### read RUN2.mdp ######
    path_to_run2=atm+folder+ "inp_files/"+"run2.mdp"    
    e=open(path_to_run2,"r")
    #print e.readlines()
    w = e.readlines()
    e.close()
    #for i in range(len(w)):
        #print i
        #print w[i]
    lines = [9, 10, 15, 16] # 7, 12, 28, 31  ]
    #for i in range(len(lines)):
        #print w[lines[i]]

    #print w[9]
    cow = w[9].split("_")
    #print cow
    #print cow[-1]
    posre_k34 = cow[-1].strip()
    #print posre_k34
        
    run2 = []
    run2.append(w[9].split())
    #run2.append(w[10].split())
    run2.append(w[15].split())
    run2.append(w[16].split())
    #print run2[0]
    #print run2[1]
    #print run2[2]
    restMDPerIter = float(run2[1][2])*float(run2[2][2])
    #print 'total restrained MD time is =', float(run2[1][2])*float(run2[2][2])
    run2mdp = []
    run2mdp.append(run2[1][2])
    run2mdp.append(run2[2][2])
    run2mdp.append(restMDPerIter)
    run2mdp.append(posre_k34)
    print run2mdp
    return run2mdp;

########################################
def get_run3mdp_data(atm,folder):

    ##### read RUN3.mdp ######
    path_to_run3=atm+folder+ "inp_files/"+"run3.mdp"    
    e=open(path_to_run3,"r")
    #print e.readlines()
    w = e.readlines()
    e.close()
    #for i in range(len(w)):
        #print i
        #print w[i]
    lines = [ 16, 17 ] # 7, 12, 28, 31  ]
    #for i in range(len(lines)):
        #print w[lines[i]]

    run3 = []
    #run3.append(w[9].split())
    #run2.append(w[10].split())
    run3.append(w[16].split())
    run3.append(w[17].split())
    #print run3[0]
    #print run3[1]
    #print run3[2]
    mdTimePerIter = float(run3[0][2])*float(run3[1][2]) 
    #print 'total unrestrained MD time is =', float(run3[0][2])*float(run3[1][2])
    run3mdp = []
    run3mdp.append(run3[0][2])
    run3mdp.append(run3[1][2])
    run3mdp.append(mdTimePerIter)
    print run3mdp
    return run3mdp; 

##########################################
def count_files(atm,folder,zipfolder):
    #print " "
    #print "path to cu"
    path_to_cu = glob.glob(atm+folder+zipfolder+"/cu009/"+"min-*_9.log")
    #print path_to_cu
    #print len(path_to_cu)
    num_minlog = int(len(path_to_cu)) #need the -1 bec it failed before completion so counter is off
    print "min log = ", num_minlog
    path_to_cu = glob.glob(atm+folder+zipfolder+"/cu009/"+"eq-*_9.log")
    num_eqlog = int(len(path_to_cu))
    print "eq log =", num_eqlog
    path_to_cu = glob.glob(atm+folder+zipfolder+"/staging/"+"md-*_1.xtc")
    num_xtc = int(len(path_to_cu))
    print "num_xtc = ", num_xtc
    path_to_cu = glob.glob(atm+folder+zipfolder+"/staging/"+"min-*_1.gro")
    num_min_gro = int(len(path_to_cu))
    print "min_gro files =", num_min_gro
    path_to_cu = glob.glob(atm+folder+zipfolder+"/staging/"+"eq-*_1.gro")
    num_eq_gro = int(len(path_to_cu))
    print "num_eq gro = ", num_eq_gro
    path_to_cu = glob.glob(atm+folder+zipfolder+"/staging/"+"coco_out*_1.gro")
    num_coco_gro = int(len(path_to_cu))
    print "num_coco gro = ", num_coco_gro
    return num_minlog, num_eqlog, num_xtc, num_min_gro, num_eq_gro, num_coco_gro;


    

In [31]:
# no need to touch
cu = 20 
iters = 20
cores = 200
#top = '../bpti/old_gits/test_git_dev/gmxcoco-bpti/inp_files/bpti.gro'
atm = '/home/feb23/Documents/jha/extasy/bpti/atmosphere_rsync/'

dirlist = ["p10_k12_00100_k34_01000_it100_right_lob_01"]

dirlist = ["p07e01_k12_00010_k34_00100", "p07e01_k12_00010_k34_00100_tx2", "p07e01_k12_00100_k34_00100", 
           "p07e01_k12_00100_k34_00100_tx2", "p07e01_k12_00100_k34_01000" , "p07e01_k12_00100_k34_01000_4d",
           "p07e01_k12_00100_k34_01000_itx2", "p07e01_k12_00100_k34_01000_mdtx2", "p07e01_k12_01000_k34_01000_it40",
          "p07e01_k12_01000_k34_01000_tx2", "p07e01_k12_10000_k34_10000", "p07e01_k12_10000_k34_10000_tx2",
          "p10_k12_00100_k34_01000_it100_right_lob_01"]

dirlist = ["p07e01_k12_00010_k34_00100", "p07e01_k12_00010_k34_00100_tx2", "p07e01_k12_00100_k34_00100", 
           "p07e01_k12_00100_k34_00100_tx2", "p07e01_k12_00100_k34_01000" , "p07e01_k12_00100_k34_01000_4d",
           "p07e01_k12_00100_k34_01000_itx2", "p07e01_k12_00100_k34_01000_mdtx2", 
          "p07e01_k12_01000_k34_01000_tx2", "p07e01_k12_10000_k34_10000", "p07e01_k12_10000_k34_10000_tx2",
          "p10_k12_00100_k34_01000_it100_right_lob_01", "p10_k12_00100_k34_01000_it48_RL",
          "p11_k12_00100_k34_01000_6d_cu80_iter10", "p07e01_k12_01000_k34_00010_4d_iter48",
          "p10_k12_00100_k34_01000_it48_RL_4d", "p07e01_k12_00010_k34_00100_4d_iter48",
          "p10_k12_00100_k34_01000_it100_right_lob_01", "p07e01_k12_00100_k34_01000_4d_iter48"]


#dirlist = ["test_p07e01_k12_00100_k34_01000_itx2"]
atm = '/home/feb23/Documents/jha/extasy/bpti/atm_sync_02_from_atm/'

dirlist = glob.glob(atm+"p*")
print dirlist
#with open('test.csv','wb') as csvfile:
#    f = csv.writer(csvfile, delimiter=" ,")
    
#f = open("tableTest01.txt","w")
#f.write("title")
#f.close()

f = open("tableTest01.txt", "w")
str1 = "dirname" + "," + "num_CUs" + "," + "approx iters completed" + ","
str2 = "r1 emstep" + "," + "r1 num steps" + "," + "r1 total time per iter" + " ," + "posre_k12" + ","
str3 = "r2 emstep" + "," + "r2 num steps" + "," + "r2 total time per iter" + " ," + "posre_k34" + ","
str4 = "r3 emstep" + "," + "r3 num steps" + "," + "r3 total time per iter" +  ","
str5 = "num_minlog" + "," + "num_eqlog" + "," + "num_xtc" +  "," 
str6 = "num_min_gro" + "," + "num_eq_gro" + "," + "num_coco_gro" + "\n"
f.write(str1+str2+str3+str4+str5+str6)


for i in range(len(dirlist)):
    print dirlist[i]
    folder=""
    #folder = dirlist[i]+"/"
    path_to_zip, rpname, zipfolder,path,pathSt, pathCu, iters, path_to_cu = set_up_vars(dirlist[i],folder)
    ## start funcs 
    cu, path_to_gmxcoco = get_gmxcoco_data(atm, folder)
    ## 
    run1mdp = get_run1mdp_data(atm,folder)
    run2mdp = get_run2mdp_data(atm,folder)
    run3mdp = get_run3mdp_data(atm,folder)
    print iters
    num_minlog, num_eqlog, num_xtc, num_min_gro, num_eq_gro, num_coco_gro = count_files(atm,folder,zipfolder)
    
    str1 = dirlist[i] + "," + str(cu) + "," + str(iters) + ","
    str2 = run1mdp[0] + "," + run1mdp[1] + "," + str(run1mdp[2]) + " ," + run1mdp[3] + ","
    str3 = run2mdp[0] + "," + run2mdp[1] + "," + str(run2mdp[2]) + " ," + run2mdp[3] + ","
    str4 = run3mdp[0] + "," + run3mdp[1] + "," + str(run3mdp[2]) + " ,"
    str5 = str(num_minlog) + "," + str(num_eqlog) + "," + str(num_xtc) + ","
    str6 = str(num_min_gro) + "," + str(num_eq_gro) + "," + str(num_coco_gro) + "\n"
    f.write(str1+str2+str3+str4+str5+str6)

f.close()

['/home/feb23/Documents/jha/extasy/bpti/atm_sync_02_from_atm/p12b03_right_d3_itrs48_k12_100_k34_1000', '/home/feb23/Documents/jha/extasy/bpti/atm_sync_02_from_atm/p12b03_right_d3_itrs48_k12_100_k34_100', '/home/feb23/Documents/jha/extasy/bpti/atm_sync_02_from_atm/p12b01_left_d3_itrs48_k12_100_k34_1000', '/home/feb23/Documents/jha/extasy/bpti/atm_sync_02_from_atm/p12b02_left_d3_itrs48_k12_1000_k34_1000', '/home/feb23/Documents/jha/extasy/bpti/atm_sync_02_from_atm/p12b03_right_d3_itrs48_k12_1000_k34_1000', '/home/feb23/Documents/jha/extasy/bpti/atm_sync_02_from_atm/p12b03_right_d4_itrs48_k12_1000_k34_1000', '/home/feb23/Documents/jha/extasy/bpti/atm_sync_02_from_atm/p12b02_left_d3_itrs48_k12_1000_k34_100', '/home/feb23/Documents/jha/extasy/bpti/atm_sync_02_from_atm/p12b03_right_d4_itrs48_k12_100_k34_1000', '/home/feb23/Documents/jha/extasy/bpti/atm_sync_02_from_atm/p12b02_left_d4_itrs48_k12_1000_k34_1000', '/home/feb23/Documents/jha/extasy/bpti/atm_sync_02_from_atm/p12b02_left_d3_itrs48_

AttributeError: 'list' object has no attribute 'split'

In [ ]:
#f = open("testfile01.csv","w")